In [ ]:
# Deepfake Detection System
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import pipeline
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import gradio as gr
import random
import time
import warnings
warnings.filterwarnings('ignore')

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============================================================================
# Neural Network Architecture
# ============================================================================

class DeepfakeDetector(nn.Module):
    """
    Convolutional Neural Network for deepfake detection
    Architecture: 4 conv layers + 3 fully connected layers
    """
    def __init__(self):
        super(DeepfakeDetector, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))

        x = x.view(-1, 256 * 14 * 14)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.sigmoid(self.fc3(x))

        return x

# Initialize model
model = DeepfakeDetector().to(device)
print("Model initialized successfully")

# ============================================================================
# AI-Powered Detection System
# ============================================================================

class AIDetectionSystem:
    """
    Advanced deepfake detection using pre-trained transformer models
    """

    def __init__(self):
        self.model_loaded = False
        self.detector = None
        self.load_model()

    def load_model(self):
        """Load pre-trained deepfake detection model"""
        try:
            self.detector = pipeline(
                "image-classification",
                model="dima806/deepfake_vs_real_image_detection",
                device=0 if torch.cuda.is_available() else -1
            )
            self.model_loaded = True
            print("AI detection model loaded successfully")

        except Exception as e:
            print(f"Warning: Could not load AI model - {e}")
            self.model_loaded = False

    def analyze_image(self, image):
        """Analyze image for deepfake indicators"""
        if not self.model_loaded:
            return None, 0.0

        try:
            result = self.detector(image)

            if isinstance(result, list) and len(result) > 0:
                label = result[0]['label'].upper()
                confidence = result[0]['score']

                is_fake = 'FAKE' in label
                return is_fake, confidence

            return None, 0.0

        except Exception as e:
            print(f"Analysis failed: {e}")
            return None, 0.0

    def get_detailed_analysis(self, image):
        """Provide detailed analysis results"""
        is_fake, confidence = self.analyze_image(image)

        if is_fake is None:
            return {
                "status": "error",
                "message": "Unable to analyze image"
            }

        # Calculate risk assessment
        if confidence > 0.8:
            certainty = "High"
        elif confidence > 0.6:
            certainty = "Medium"
        else:
            certainty = "Low"

        return {
            "status": "success",
            "prediction": "Synthetic" if is_fake else "Authentic",
            "confidence": confidence,
            "certainty_level": certainty,
            "risk_score": confidence if is_fake else (1 - confidence)
        }

# Initialize AI system
ai_system = AIDetectionSystem()

# ============================================================================
# Image Processing Pipeline
# ============================================================================

def process_image(image_input):
    """
    Standardize image input for analysis
    Supports: PIL Image, numpy array, file path
    """
    try:
        if isinstance(image_input, Image.Image):
            image = image_input
        elif isinstance(image_input, np.ndarray):
            if len(image_input.shape) == 3 and image_input.shape[2] == 3:
                image_input = cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image_input)
        elif isinstance(image_input, str):
            image = Image.open(image_input)
        else:
            raise ValueError("Unsupported image format")

        # Ensure RGB format
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Resize to standard dimensions
        image = image.resize((224, 224), Image.Resampling.LANCZOS)

        return image

    except Exception as e:
        print(f"Image processing error: {e}")
        return None

def create_tensor(image):
    """Convert PIL image to PyTorch tensor"""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

# ============================================================================
# Main Detection Function
# ============================================================================

def detect_deepfake(image_input, detection_method="AI Model"):


    # Process input image
    processed_image = process_image(image_input)
    if processed_image is None:
        return {"error": "Failed to process image"}

    results = {
        "image_processed": True,
        "dimensions": processed_image.size,
        "method": detection_method
    }

    if detection_method == "AI Model":
        # Use AI-powered detection
        analysis = ai_system.get_detailed_analysis(processed_image)

        if analysis["status"] == "success":
            results.update({
                "prediction": analysis["prediction"],
                "confidence": analysis["confidence"],
                "certainty": analysis["certainty_level"],
                "risk_score": analysis["risk_score"]
            })
        else:
            results["error"] = analysis["message"]

    elif detection_method == "Basic CNN":
        # Use basic CNN model (note: untrained in this demo)
        tensor_img = create_tensor(processed_image).to(device)

        with torch.no_grad():
            output = model(tensor_img)
            prediction = output.item()

        results.update({
            "prediction": "Synthetic" if prediction > 0.5 else "Authentic",
            "confidence": abs(prediction - 0.5) * 2,
            "certainty": "Demo only",
            "note": "Basic model is for demonstration purposes"
        })

    return results

# ============================================================================
# User Interface
# ============================================================================

def create_professional_interface():

    def analyze_uploaded_image(image, method):
       =
        if image is None:
            return "Please upload an image to analyze."

        try:
            results = detect_deepfake(image, method)

            if "error" in results:
                return f"Analysis Error: {results['error']}\n\nPlease try uploading a different image."

            # Format results professionally
            if method == "AI Model":
                output = f"""Analysis Complete

Prediction: {results['prediction']}
Confidence: {results['confidence']:.1%}
Certainty Level: {results['certainty']}

Technical Details:
• Image dimensions: {results['dimensions'][0]} × {results['dimensions'][1]} pixels
• Detection method: {results['method']}
• Processing time: < 1 second

Note: This tool uses state-of-the-art AI models for deepfake detection.
Results should be considered as part of a broader verification process."""

            else:
                output = f"""Analysis Complete

Prediction: {results['prediction']}
Confidence: {results['confidence']:.1%}

Technical Details:
• Image dimensions: {results['dimensions'][0]} × {results['dimensions'][1]} pixels
• Detection method: {results['method']}

{results.get('note', '')}

Note: Basic CNN model is for demonstration purposes only."""

            return output

        except Exception as e:
            return f"System Error: {str(e)}\n\nPlease try again or contact support."

    # Create interface
    interface = gr.Interface(
        fn=analyze_uploaded_image,
        inputs=[
            gr.Image(
                type="pil",
                label="Upload Image",
                height=400
            ),
            gr.Radio(
                choices=["AI Model", "Basic CNN"],
                value="AI Model",
                label="Detection Method",
                info="AI Model uses pre-trained transformers (recommended). Basic CNN is for demonstration."
            )
        ],
        outputs=gr.Textbox(
            label="Analysis Results",
            lines=12,
            max_lines=15
        ),
        title="Deepfake Detection System",
        description="""
        Upload an image to analyze whether it's authentic or AI-generated.

        This tool uses advanced machine learning models to detect potential deepfakes.
        Best results are achieved with clear, high-quality images containing faces.

        **Important:** This is a research tool. Results should be verified through additional methods.
        """,
        theme=gr.themes.Soft(),
        allow_flagging="never",
        css="""
        .gradio-container {
            max-width: 800px !important;
            margin: auto !important;
        }
        .gr-button {
            background: linear-gradient(90deg, #4f46e5, #7c3aed) !important;
            border: none !important;
        }
        .gr-button:hover {
            transform: translateY(-1px) !important;
        }
        """
    )

    return interface

# ============================================================================
# System Diagnostics
# ============================================================================

def run_system_check():
    """Verify system components are working"""
    print("Running system diagnostics...")

    # Test image processing
    test_image = Image.new('RGB', (224, 224), color=(128, 128, 128))
    processed = process_image(test_image)

    if processed is not None:
        print("✓ Image processing: OK")
    else:
        print("✗ Image processing: Failed")

    # Test AI detection
    if ai_system.model_loaded:
        print("✓ AI detection model: Loaded")
    else:
        print("✗ AI detection model: Not available")

    # Test basic model
    try:
        tensor_img = create_tensor(test_image).to(device)
        with torch.no_grad():
            _ = model(tensor_img)
        print("✓ Basic CNN model: OK")
    except Exception as e:
        print(f"✗ Basic CNN model: Error - {e}")

    print("System check complete")

# ============================================================================
# Application Launch
# ============================================================================

def launch_application():
    """Launch the deepfake detection application"""
    print("Deepfake Detection System")
    print("=" * 40)

    # Run diagnostics
    run_system_check()

    print("\nLaunching web interface...")

    # Create and launch interface
    interface = create_professional_interface()

    interface.launch(
        share=True,
        server_name="0.0.0.0",
        show_error=True,
        quiet=False
    )

# ============================================================================
# Additional Utilities
# ============================================================================

def get_system_info():
    """Display system information"""
    info = {
        "PyTorch Version": torch.__version__,
        "Device": str(device),
        "CUDA Available": torch.cuda.is_available(),
        "AI Model Status": "Loaded" if ai_system.model_loaded else "Not Available",
        "Model Parameters": f"{sum(p.numel() for p in model.parameters()):,}"
    }

    return info

def analyze_image_metadata(image_path):
    """Extract and analyze image metadata"""
    try:
        from PIL.ExifTags import TAGS
        image = Image.open(image_path)

        metadata = {
            "filename": image_path,
            "format": image.format,
            "mode": image.mode,
            "size": image.size,
            "has_exif": bool(image._getexif())
        }

        return metadata

    except Exception as e:
        return {"error": str(e)}

# Display system information
print("\nSystem Information:")
for key, value in get_system_info().items():
    print(f"  {key}: {value}")

# ============================================================================
# Main Execution
# ============================================================================

if __name__ == "__main__":
    launch_application()
else:
    print("Deepfake Detection System loaded successfully")
    print("Available functions:")
    print("  - detect_deepfake(image, method)")
    print("  - analyze_image_metadata(image_path)")
    print("  - get_system_info()")
    print("  - launch_application()")

Using device: cuda
Model initialized successfully


Device set to use cuda:0


AI detection model loaded successfully

System Information:
  PyTorch Version: 2.6.0+cu124
  Device: cuda
  CUDA Available: True
  AI Model Status: Loaded
  Model Parameters: 26,144,833
Deepfake Detection System
Running system diagnostics...
✓ Image processing: OK
✓ AI detection model: Loaded
✓ Basic CNN model: OK
System check complete

Launching web interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ba308403d82f3350b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
